# Cyclist going around Map04

In [1]:
# boilerplate code
import carla 
import math 
import random 
import time 
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Create a file handler
file_handler = logging.FileHandler('../data/carla_simulation.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))

# Add the file handler to the logger
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
world = client.get_world() 

# load Town04 map
world = client.load_world('Town04')

# Get the blueprint library and the spawn points for the map
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

# Get the blueprint for the bycicle you want
vehicle_bp = bp_lib.find('vehicle.bh.crossbike') 
# nb other options are:
# 'vehicle.gazelle.omafiets',
#vehicle_bp = bp_lib.find('vehicle.diamondback.century') 
# 'vehicle.diamondback.century'

# Try spawning the vehicle at a randomly chosen spawn point
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# start the simulation
vehicle.set_autopilot(True)



## Following vehicle with a camera

In [2]:
from carla_camera import CameraManager

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# Get spectator
spectator = world.get_spectator()

# Spawn vehicle (example)
spawn_points = world.get_map().get_spawn_points()
# vehicle_bp = world.get_blueprint_library().find('vehicle.tesla.model3')
vehicle = world.spawn_actor(vehicle_bp, spawn_points[0])
vehicle.set_autopilot(True)

# Create camera manager
camera = CameraManager()

# Cell 2: Control Camera
# Start following
camera.start_following(world, vehicle, spectator)

Camera following started


## Modify views on the fly
This can be done as long as logging loop is not running

In [12]:
# camera.set_front_view()
#camera.set_front_right_diagonal()
#camera.set_back_view_tilted()
#camera.set_front_left_diagonal()
camera.set_custom_view(-20, 0, 10, -6, 0, 0)


## Monitoring Loop

In [16]:
# Cell 2: Monitoring Loop
try:
    # Main loop to monitor steering angle
    while True:
        # Get vehicle control
        control = vehicle.get_control()
        
        # Get steering angle (ranges from -1.0 to 1.0)
        steering_angle = control.steer
        
        # Convert to degrees for better readability (-70 to 70 degrees)
        steering_degrees = steering_angle * 70.0
        
        # Print steering information with different formats
        logger.info(f"Steering Angle: Raw={steering_angle:.3f}, Degrees={steering_degrees:.1f}°")
        
        # Get vehicle's transform for additional context
        transform = vehicle.get_transform()
        logger.info(f"Vehicle Location: x={transform.location.x:.1f}, "
                   f"y={transform.location.y:.1f}, "
                   f"heading={transform.rotation.yaw:.1f}°")
        
        # Add a small delay to prevent flooding the output
        time.sleep(0.1)
        
except KeyboardInterrupt:
    logger.info("Monitoring stopped by user")
finally:
    # Cleanup
    vehicle.destroy()
    logger.info("Vehicle destroyed")

2025-01-18 16:39:42,491 - Steering Angle: Raw=0.000, Degrees=0.0°
2025-01-18 16:39:42,493 - Vehicle Location: x=315.7, y=-347.2, heading=29.4°
2025-01-18 16:39:42,594 - Steering Angle: Raw=-0.000, Degrees=-0.0°
2025-01-18 16:39:42,595 - Vehicle Location: x=316.1, y=-346.9, heading=29.4°
2025-01-18 16:39:42,698 - Steering Angle: Raw=-0.001, Degrees=-0.1°
2025-01-18 16:39:42,699 - Vehicle Location: x=316.5, y=-346.7, heading=29.4°
2025-01-18 16:39:42,802 - Steering Angle: Raw=-0.001, Degrees=-0.1°
2025-01-18 16:39:42,803 - Vehicle Location: x=316.9, y=-346.5, heading=29.5°
2025-01-18 16:39:42,905 - Steering Angle: Raw=-0.001, Degrees=-0.1°
2025-01-18 16:39:42,907 - Vehicle Location: x=317.3, y=-346.2, heading=29.4°
2025-01-18 16:39:43,010 - Steering Angle: Raw=0.058, Degrees=4.1°
2025-01-18 16:39:43,011 - Vehicle Location: x=317.8, y=-346.0, heading=30.3°
2025-01-18 16:39:43,113 - Steering Angle: Raw=0.009, Degrees=0.6°
2025-01-18 16:39:43,115 - Vehicle Location: x=318.1, y=-345.8, headi

: 

## Stop the spectator from following vehicle

In [15]:
camera.stop_following()

Camera following stopped


## Start spectator vehicle following

In [14]:
camera.start_following(world, vehicle, spectator)

Camera following started


## Youtube video capture
https://youtu.be/A7Wn2t2cFaM